###  Imports

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

pd.set_option('display.max_rows', 500)

In [2]:
column_order = ['dataset used', 'classifier', 'feature used', 'min', 'max', 'median', 'mean']
feature_list = ['phase', 'breathing', 'heartbeat', 'combined br hb', 'undercomplete', 'sparse', 'deep']
results_path = "results/"

In [3]:
def make_an_aggregated_dataframe(dataset_used, feature_used, accs_dataframe):
    
    # Initialize a dataframe
    return_df = pd.DataFrame()
    
    # Initialize a dictionary
    stats_dict = {}
    stats_dict['dataset used'] = dataset_used
    stats_dict['feature used'] = feature_used
    
    # Loop over all classifiers and 
    for classifier in accs_dataframe.index:
        
        # Extract the accuracies array
        accs_array = np.array(accs_dataframe.loc[classifier])
        
        # Calculate statistics
        stats_dict['classifier'] = classifier
        stats_dict['min'] = np.min(accs_array)
        stats_dict['max'] = np.max(accs_array)
        stats_dict['median'] = np.median(accs_array)
        stats_dict['mean'] = np.mean(accs_array)
#         stats_dict['data_len'] = len(accs_array)
        
        # Append to dataframe
        return_df = return_df.append(stats_dict, ignore_index=True)
    
    return_df = return_df[column_order]
    
    return return_df

### Busy vs Relaxed

In [4]:
BvR_df = pd.DataFrame()
BvR_prefix = "BvR/BvR"
dataset_used_midpoint = "-all"
tfv1_file_ending = "-tfv1"
file_format_ending = ".csv"

In [5]:
df_all = pd.read_csv(results_path + BvR_prefix + dataset_used_midpoint + file_format_ending, index_col=[0])

for feature_used in feature_list:
    temp_df = df_all[feature_used].apply(literal_eval)
    temp_df = make_an_aggregated_dataframe('all', feature_used, temp_df)
    BvR_df = BvR_df.append(temp_df, ignore_index=True)

In [6]:
df_all_tfv1 = pd.read_csv(results_path + BvR_prefix + dataset_used_midpoint + tfv1_file_ending + file_format_ending, index_col=[0])
temp_df = df_all_tfv1['contractive'].apply(literal_eval)
temp_df = make_an_aggregated_dataframe('all', 'contractive', temp_df)
BvR_df = BvR_df.append(temp_df, ignore_index=True)

In [7]:
BvR_prefix = "BvR/BvR"
dataset_used_midpoint = "-NC"
tfv1_file_ending = "-tfv1"
file_format_ending = ".csv"

In [8]:
df_NC = pd.read_csv(results_path + BvR_prefix + dataset_used_midpoint + file_format_ending, index_col=[0])

for feature_used in feature_list:
    temp_df = df_NC[feature_used].apply(literal_eval)
    temp_df = make_an_aggregated_dataframe('NC', feature_used, temp_df)
    BvR_df = BvR_df.append(temp_df, ignore_index=True)

In [9]:
df_NC_tfv1 = pd.read_csv(results_path + BvR_prefix + dataset_used_midpoint + tfv1_file_ending + file_format_ending, index_col=[0])
temp_df = df_NC_tfv1['contractive'].apply(literal_eval)
temp_df = make_an_aggregated_dataframe('NC', 'contractive', temp_df)
BvR_df = BvR_df.append(temp_df, ignore_index=True)

In [10]:
BvR_df.sort_values(['dataset used', 'classifier'], ascending=[True,True], inplace=True)
BvR_df

,dataset used,classifier,feature used,min,max,median,mean
57,NC,LSTM,phase,0.250000,1.000000,0.500000,0.642157
64,NC,LSTM,breathing,0.500000,1.000000,0.500000,0.696078
71,NC,LSTM,heartbeat,0.000000,1.000000,0.500000,0.485294
78,NC,LSTM,combined br hb,0.500000,1.000000,0.666667,0.700980
85,NC,LSTM,undercomplete,0.250000,1.000000,0.500000,0.544118
92,NC,LSTM,sparse,0.000000,1.000000,0.500000,0.544118
99,NC,LSTM,deep,0.000000,1.000000,0.500000,0.500000
106,NC,LSTM,contractive,0.166667,1.000000,0.500000,0.534314
59,NC,SVC,phase,0.500000,1.000000,0.666667,0.661765
66,NC,SVC,breathing,0.500000,1.000000,0.833333,0.769608


In [11]:
BvR_df.loc[BvR_df['classifier'] == 'LSTM']

,dataset used,classifier,feature used,min,max,median,mean
57,NC,LSTM,phase,0.250000,1.000,0.500000,0.642157
64,NC,LSTM,breathing,0.500000,1.000,0.500000,0.696078
71,NC,LSTM,heartbeat,0.000000,1.000,0.500000,0.485294
78,NC,LSTM,combined br hb,0.500000,1.000,0.666667,0.700980
85,NC,LSTM,undercomplete,0.250000,1.000,0.500000,0.544118
92,NC,LSTM,sparse,0.000000,1.000,0.500000,0.544118
99,NC,LSTM,deep,0.000000,1.000,0.500000,0.500000
106,NC,LSTM,contractive,0.166667,1.000,0.500000,0.534314
1,all,LSTM,phase,0.500000,1.000,0.750000,0.718460
8,all,LSTM,breathing,0.437500,1.000,0.700000,0.724836


In [12]:
BvR_df.to_csv(results_path + "BvR.csv", mode='w', index=False)